In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import isfile, join
import datetime
import timeit


IN_LOG_TIME_FORMAT = '%Y-%m-%d %H:%M:%S.%f'
convert2Time = lambda x: datetime.datetime.strptime(x, IN_LOG_TIME_FORMAT)
findClosestTime = lambda timeStamp, vector : abs(vector - timeStamp).idxmin()

In [34]:
path = 'C:\\Users\\NVIDIA\\abstract-fps\\code\\data-files\\result_data\\simpleMotion\\KB-3-26\\'
path = 'C:\\Users\\NVIDIA\\abstract-fps\\code\\results\\KamranPilot\\'
path = 'C:\\Users\\NVIDIA\\abstract-fps\\code\\results\\'
path = r'C:\Users\\NVIDIA\Documents\\Python Scripts\\data\\Alpha_JK\\'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
print('The file Names:\n ', onlyfiles)

The file Names:
  ['target_JK_2019_04_12-15_41_24.db', 'target_JK_2019_04_12-15_59_46.db', 'target_JK_2019_04_12-16_09_40.db', 'target_JK_2019_04_12-16_32_53.db', 'target_JK_2019_04_12-16_36_10.db', 'target_JK_2019_04_12-16_41_24.db', 'target_JK_2019_04_12-16_44_00.db', 'target_JK_2019_04_12-16_49_49.db', 'target_JK_2019_04_12-16_57_55.db', 'target_JK_2019_04_12-17_04_22.db', 'target_JK_2019_04_12-17_07_41.db', 'target_JK_2019_04_12-17_13_43.db', 'target_JK_2019_04_12-17_16_57.db', 'target_JK_2019_04_12-17_22_56.db', 'target_JK_2019_04_12-17_26_06.db', 'target_JK_2019_04_12-17_34_09.db', 'target_JK_2019_04_12-17_37_50.db', 'target_JK_2019_04_12-17_43_23.db', 'target_JK_2019_04_12-17_46_33.db']


In [36]:
# Select the subject ID
mySubject = 'JK'

conditionDF = []
trialDF = []
targetDF = []
playerDF = []
experimentDF = []
clickLatencyDF = []
eventLogDF = []
IVPairs = []

conditionIdx = 0
trialIdx = 0
targetIdx = 0
playerIdx = 0
experimentIdx = 0

for fileName in onlyfiles:
    #print(fileName)
    dat = sqlite3.connect(path + fileName)
    cursor = dat.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    condition = pd.read_sql_query("SELECT * FROM Conditions", dat)
    experiment = pd.read_sql_query("SELECT * FROM Experiments", dat)    
    if (experiment['subjectID'].values != mySubject):
        continue
    if len(condition) == 0:
        continue
    print(fileName)
    #print(cursor.fetchall())
    trials = pd.read_sql_query("SELECT * FROM Trials", dat)
    trials['refresh_rate'] = condition.refresh_rate.values[0]
    latencyMS = np.round(1000*condition.added_frame_lag.values[0]/condition.refresh_rate.values[0],0)
    #print(condition.added_frame_lag.values[0],condition.refresh_rate.values[0], latencyMS)
    condition['added_frame_lag'] = latencyMS
    trials['added_frame_lag'] = latencyMS
    target = pd.read_sql_query("SELECT * FROM Target_Trajectory", dat)
    target['position_az'] = np.arctan2(-target.position_z.values,-target.position_x.values)* 180 / np.pi
    target['position_el'] = np.arctan2(target.position_y.values,np.sqrt(np.power(target.position_x.values,2) + np.power(target.position_z.values,2))) * 180 / np.pi
    target['refresh_rate'] = condition.refresh_rate.values[0]
    target['added_frame_lag'] = latencyMS
    player = pd.read_sql_query("SELECT * FROM Player_Action", dat)
    player['refresh_rate'] = condition.refresh_rate.values[0]
    player['added_frame_lag'] = latencyMS
    clickLatency = pd.read_sql_query("SELECT * FROM click_latencies", dat)
    eventLog = pd.read_sql_query("SELECT * FROM event_log", dat)
    
    IVPairs.append([condition.refresh_rate.values[0], latencyMS])
    #condition['global_index'] = conditionIdx + np.arange(len(condition))
    #conditionIdx = conditionIdx + len(condition)
    #trials['global_index'] = trialIdx + np.arange(len(trials))
    #trialIdx = trialIdx + len(trials)
    #target['global_index'] = targetIdx + np.arange(len(target))
    #targetIdx = targetIdx + len(target)
    #player['global_index'] = playerIdx + np.arange(len(player))
    #playerIdx = playerIdx + len(player)
    #experiment['global_index'] = experimentIdx + np.arange(len(experiment))
    #experimentIdx = experimentIdx + len(experiment)
    
    
    conditionDF.append(condition)
    trialDF.append(trials)
    targetDF.append(target)
    playerDF.append(player)
    experimentDF.append(experiment)
    clickLatencyDF.append(clickLatency)
    eventLogDF.append(eventLog)
    
allConditions = pd.concat(conditionDF, ignore_index=True)
allTrials = pd.concat(trialDF, ignore_index=True)
allTargets = pd.concat(targetDF, ignore_index=True)
allPlayers = pd.concat(playerDF, ignore_index=True)
allExperiments = pd.concat(experimentDF, ignore_index=True)
allClickLatency = pd.concat(clickLatencyDF, ignore_index=True)
allEventLog = pd.concat(eventLogDF, ignore_index=True)
#allConditions.reindex(range(len(allConditions)))
#allTrials.reindex('global_index')
#allTargets.reindex('global_index')
#allPlayers.reindex('global_index')
#allExperiments.reindex('global_index')


#allConditions.indexes = allConditions.global_index.values
#allTrials.indexes = allTrials.global_index.values
#allTargets.indexes = allTargets.global_index.values
#allPlayers.indexes = allPlayers.global_index.values
#allExperiments.indexes = allExperiments.global_index.values

target_JK_2019_04_12-15_41_24.db
target_JK_2019_04_12-15_59_46.db
target_JK_2019_04_12-16_09_40.db
target_JK_2019_04_12-16_32_53.db
target_JK_2019_04_12-16_36_10.db
target_JK_2019_04_12-16_41_24.db
target_JK_2019_04_12-16_44_00.db
target_JK_2019_04_12-16_49_49.db
target_JK_2019_04_12-16_57_55.db
target_JK_2019_04_12-17_04_22.db
target_JK_2019_04_12-17_07_41.db
target_JK_2019_04_12-17_13_43.db
target_JK_2019_04_12-17_16_57.db
target_JK_2019_04_12-17_22_56.db
target_JK_2019_04_12-17_26_06.db
target_JK_2019_04_12-17_34_09.db
target_JK_2019_04_12-17_37_50.db
target_JK_2019_04_12-17_43_23.db
target_JK_2019_04_12-17_46_33.db


In [37]:
allPlayers['time'] = allPlayers['time'].apply(convert2Time)
allTargets['time'] = allTargets['time'].apply(convert2Time)
allTrials['start_time'] = allTrials['start_time'].apply(convert2Time)
allTrials['end_time'] = allTrials['end_time'].apply(convert2Time)
allClickLatency['time'] = allClickLatency['time'].apply(convert2Time)

In [38]:
def findIndexInPlayerDF(allClickLatency, allPlayers):
    myArray = []
    for idx in range(len(allClickLatency)):
        timeStamp = allClickLatency['time'].values[idx]
        myArray.append(np.argmin(abs(allPlayers['time'].values - timeStamp)))
    allClickLatency['player_global_Idx'] = np.array(myArray)
#allClickLatency['player_global_Idx'] = allClickLatency['time'].apply(findClosestTime, vector = allPlayers['time'])

In [39]:
findIndexInPlayerDF()

In [40]:
allClickLatency

,time,latency,player_global_Idx
0,2019-04-12 15:51:25.110301,16.212,0
1,2019-04-12 15:51:28.416245,17.276,0
2,2019-04-12 15:51:28.821357,26.580,143
3,2019-04-12 15:51:29.615617,16.648,144
4,2019-04-12 15:51:29.948065,17.944,262
5,2019-04-12 15:51:30.210137,17.316,357
6,2019-04-12 15:51:30.720845,29.488,538
7,2019-04-12 15:51:31.588713,18.260,540
8,2019-04-12 15:51:31.799805,18.568,540
9,2019-04-12 15:51:32.507745,14.312,794


In [41]:
allClickLatency['refresh_rate'] = allPlayers['refresh_rate'].values[allClickLatency['player_global_Idx'].values]
allClickLatency['added_frame_lag'] = allPlayers['added_frame_lag'].values[allClickLatency['player_global_Idx'].values]


In [42]:
print(allClickLatency.added_frame_lag.unique())
print(allClickLatency.refresh_rate.unique())

[ 6.  8.  0. 56. 58. 50. 92.]
[360. 240. 120.  60.]


In [10]:
#allClickLatency.added_frame_lag.unique()
#allClickLatency['added_frame_lag'].values[allClickLatency['added_frame_lag'].values < 50] = 0
#allClickLatency['added_frame_lag'].values[allClickLatency['added_frame_lag'].values == 56] = 50
#allClickLatency['added_frame_lag'].values[allClickLatency['added_frame_lag'].values == 58] = 50


In [43]:
#allClickLatency = allClickLatency.drop(allClickLatency.iloc[allClickLatency.added_frame_lag.values == 92.0].index)
trialsGB = allClickLatency.groupby(['refresh_rate', 'added_frame_lag'])

refreshRates = allClickLatency.refresh_rate.unique()
latencies = allClickLatency.added_frame_lag.unique()
#latencies = np.array([0, 50])
figureFileName = 'KB-4-7'
plt.figure(figsize = (10,10))

#colors = dict(dict(zip(latencies, ['g', 'c', 'r', 'b', 'k', 'y'])))
colors = dict(dict(zip(latencies, ['g', 'c'])))#, 'r', 'b', 'k', 'y'])))
#markers = dict(dict(zip(latencies, ['-o', '-^', '-D', '-o', '-^', '-D'])))
markers = dict(dict(zip(latencies, ['-o', '-^'])))#, '-D', '-o', '-^', '-D'])))
#offset = dict(dict(zip(latencies, [-0.1, 0., 0.1, -0.2, 0.2, 0.3])))
offset = dict(dict(zip(latencies, [-0.1, 0.])))#, 0.1, -0.2, 0.2, 0.3])))

print('Refresh Rates :', refreshRates)
print('Latencies :', latencies)


for (rf, lt) in trialsGB.groups.keys():
    #print(IVs)
    currentDF = trialsGB.get_group((rf,lt))
    reactionTimes = np.array(currentDF.latency.values)
    plt.figure(figsize = (10,8))
    num_bins = 50
    # the histogram of the data
    n, bins, patches = plt.hist(reactionTimes, num_bins, density=True, facecolor='green', edgecolor = 'black',alpha=0.5)

    # add a 'best fit' line
    #y = mlab.normpdf(bins, mu, sigma)
    #plt.plot(bins, y, 'r--')
    plt.xlabel('time [ms]', fontsize = 16)
    plt.ylabel('Probability', fontsize = 16)
    plt.title(str(rf)+'_'+str(lt)+'_'+mySubject, fontsize = 16)
    plt.grid(True) 

    # Tweak spacing to prevent clipping of ylabel
    #plt.subplots_adjust(left=0.15)
    #plt.text(-0.08, 99, 'Latency:'+lt+'ms, Subject '+ mySubject, fontsize = 16)
    plt.savefig('clickToPhoton_'+mySubject+'_'+str(lt)+'_'+str(rf)+'.png', dpi = 200)
    #plt.show()
    plt.close()


Refresh Rates : [360. 240. 120.  60.]
Latencies : [ 6.  8.  0. 56. 58. 50. 92.]


<Figure size 720x720 with 0 Axes>

In [20]:
trialsGB.groups.keys()

dict_keys([(60.0, 0.0), (60.0, 50.0), (120.0, 8.0), (120.0, 58.0), (240.0, 8.0), (240.0, 58.0)])

In [19]:
allClickLatency = allClickLatency.drop(allClickLatency.iloc[allClickLatency.added_frame_lag.values == 92.0].index)
trialsGB = allClickLatency.groupby(['refresh_rate', 'added_frame_lag'])

refreshRates = allClickLatency.refresh_rate.unique()
latencies = allClickLatency.added_frame_lag.unique()
latencies = np.array([0, 50])
figureFileName = 'KB-4-7'
plt.figure(figsize = (10,10))

#colors = dict(dict(zip(latencies, ['g', 'c', 'r', 'b', 'k', 'y'])))
colors = dict(dict(zip(latencies, ['g', 'c'])))#, 'r', 'b', 'k', 'y'])))
#markers = dict(dict(zip(latencies, ['-o', '-^', '-D', '-o', '-^', '-D'])))
markers = dict(dict(zip(latencies, ['-o', '-^'])))#, '-D', '-o', '-^', '-D'])))
#offset = dict(dict(zip(latencies, [-0.1, 0., 0.1, -0.2, 0.2, 0.3])))
offset = dict(dict(zip(latencies, [-0.1, 0.])))#, 0.1, -0.2, 0.2, 0.3])))

print('Refresh Rates :', refreshRates)
print('Latencies :', latencies)

for lt in latencies:
    for rf in refreshRates:
    
        #print(IVs)
        currentDF = trialsGB.get_group((rf,lt))
        reactionTimes = np.array(currentDF.latency.values)
        mean = np.nanmean(reactionTimes)
        std = np.nanstd(reactionTimes)
        array = np.append(array,[mean,std])
    #print(array)
    plt.errorbar(np.arange(len(refreshRates)) + np.array(offset[lt]), array[0:-1:2], yerr= array[1::2], fmt=markers[lt], ecolor=colors[lt], 
                 capsize=6, markersize = 8, linewidth = 3, color = colors[lt], elinewidth = 3, label = str(int(lt))+' (ms)')

plt.grid(True)
plt.xticks(np.arange(len(refreshRates)), (refreshRates), fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('Click to photon latency', fontsize = 16)
plt.legend(fontsize = 14)
#plt.axis('equal')
plt.xlabel('Refresh Rate (fps)', fontsize = 14)
plt.ylabel('Latency (s)', fontsize = 14)
plt.text(-0.08, 99, 'Subject '+ mySubject, fontsize = 16)
plt.ylim(5, 110)
plt.savefig('clickToPhoton_'+mySubject+'.png', dpi = 200)
plt.show()

Refresh Rates : [240. 120.  60.]
Latencies : [ 0 50]


KeyError: (240.0, 0)

<Figure size 720x720 with 0 Axes>

In [ ]:
trialsGB.groups.keys()

In [ ]:
allClickLatency.added_frame_lag.unique()

In [ ]:
np.arange(len(refreshRates)) + np.array(offset[lt])

In [ ]:
trialsGB = allClickLatency.groupby(['refresh_rate', 'added_frame_lag'])
trialsGB.groups.keys()

In [ ]:
trialsGB = allClickLatency.groupby(['refresh_rate', 'added_frame_lag'])
trialsGB.groups.keys()

In [ ]:
allClickLatency.iloc[allClickLatency.added_frame_lag.values == 92.0].index

In [ ]:
allClickLatency = allClickLatency.drop(allClickLatency.iloc[allClickLatency.added_frame_lag.values == 92.0].index)
newDF.added_frame_lag.unique()